# RNN Module

rnn 버리고 lstm으로 먼저 구현해봅니다 <br />
input : FormattedReview List <br />
output : numpy array <br />

FormattedReview
- context              : Numpy array
- context_bayes        : Tuple List
- comp_similarity      : float
- rate                 : float
- reiteration_context  : float
- reiteration_repeat   : float
- post_time            : float
- post_vip             : float
- id                   : float

- index                : int
- label                : bool

sample code from <br />
https://discuss.pytorch.org/t/example-of-many-to-one-lstm/1728/2

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable    

In [ ]:
model = nn.LSTM(in_size, classes_no, 2)
input_seq = Variable(torch.randn(time_steps, batch_size, in_size))
output_seq, _ = model(input_seq)
last_output = output_seq[-1]

loss = nn.CrossEntropyLoss()
target = Variable(torch.LongTensor(batch_size).random_(0, classes_no-1))
err = loss(last_output, target)
err.backward()

In [18]:
input = Variable(torch.randn(3, 1, 5))
h0 = Variable(torch.randn(1, 1, 10))
c0 = Variable(torch.randn(1, 1, 10))

In [38]:
rnn = nn.LSTM(5, 10, 1)
output, hn = rnn(input, (h0, c0))
print(input)
print(output)

Variable containing:
(0 ,.,.) = 
 -0.1007  0.0854 -0.2965  0.1249 -1.0662

(1 ,.,.) = 
  0.6835  1.2083 -0.4033 -0.8664  0.4078

(2 ,.,.) = 
 -0.0301 -0.4181  0.0801  0.8633  1.7904
[torch.FloatTensor of size 3x1x5]

Variable containing:
(0 ,.,.) = 

Columns 0 to 8 
   0.3721 -0.2455  0.4047  0.0188 -0.0347 -0.1264  0.0526 -0.1881 -0.1872

Columns 9 to 9 
  -0.1837

(1 ,.,.) = 

Columns 0 to 8 
   0.2394 -0.0665  0.3482  0.1626 -0.0491 -0.1298  0.0904 -0.0084 -0.2787

Columns 9 to 9 
  -0.2293

(2 ,.,.) = 

Columns 0 to 8 
   0.2003 -0.0439  0.2087 -0.0600  0.1815  0.0176  0.0824  0.0505 -0.0520

Columns 9 to 9 
  -0.0149
[torch.FloatTensor of size 3x1x10]



In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
    
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]
    
sys.meta_path.append(NotebookFinder())

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable    

input_size = 100  # word2vec k size
hidden_size = 10  # hyperparameter
num_layers = 3    # hyperparameter
p_dropout = 0.05  # hyperparameter

### sort_batch from
https://discuss.pytorch.org/t/about-the-variable-length-input-in-rnn-scenario/345/22

In [13]:
class RNN_model(nn.Module):
    def __init__(self):
        super(RNN_model, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.num_layers = num_layers
        
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, dropout = p_dropout)
        
    def encoder(self, formattedList):
        length = len(formattedList)
        contextList = [formattedList[i].context for i in range(length)]

        lengths = torch.LongTensor([len(contextList[i]) for i in range(length)])
        max_len = torch.max(lengths)

        data = np.zeros((length, max_len, input_size))

        for i in range(length):
            context = contextList[i]
            if not (context.size == 0):
                data[i, :context.shape[0],:] = context
            else:
                lengths[i] = 1
            i+=1
            
        return self.sort_batch(torch.FloatTensor(data), lengths)
        
    def sort_batch(self, data, seq_len):
        batch_size = data.size(0)
        sorted_seq_len, sorted_idx = seq_len.sort()
        reverse_idx = torch.linspace(batch_size-1,0,batch_size).long()
        sorted_seq_len = sorted_seq_len[reverse_idx]
        sorted_data = data[sorted_idx][reverse_idx]
        
        return sorted_data, sorted_seq_len
        
    def forward(self, formattedList, hidden=None):
        
        embeddings, lengths = self.encoder(formattedList)
        
        packed = nn.utils.rnn.pack_padded_sequence(Variable(embeddings), lengths.tolist(), batch_first=True)
        output, hidden = self.rnn(packed, hidden)
        output, _ = nn.utils.rnn.pad_packed_sequence(output, batch_first=True)
        
        idx = (lengths-1).view(-1,1).expand(output.size(0), output.size(2)).unsqueeze(1)
        
        print(output)
        decoded = output.gather(1, Variable(idx)).squeeze()
        
        return decoded

In [15]:
import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):
    with open(filename, 'rb') as input:
        return pickle.load(input)

In [5]:
import format_module
import bookingreview
import preprocessreview
import wordvectormaker
import postag_module
import gensim
import xl_to_br_module

importing Jupyter notebook from format_module.ipynb


C:\Program Files\Anaconda\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [14]:
formatted_list = load_object("save_formatted_review.pkl")
test_rnn = RNN_model()

print(formatted_list[0].context.shape)
print(test_rnn(formatted_list), "\n\n")

for param in test_rnn.parameters():
     print(type(param.data), param.size())

(120, 100)
Variable containing:
( 0 ,.,.) = 
 -0.0461  0.0323  0.0099  ...   0.1029  0.0343 -0.0305
 -0.0491  0.0480  0.0247  ...   0.1494  0.0522 -0.0513
 -0.0429  0.0605  0.0236  ...   0.1655  0.0645 -0.0767
           ...             ⋱             ...          
 -0.0270  0.0444  0.0072  ...   0.1774  0.0445 -0.0925
 -0.0370  0.0427  0.0188  ...   0.1811  0.0445 -0.0966
 -0.0418  0.0470 -0.0020  ...   0.1629  0.0441 -0.1115

( 1 ,.,.) = 
 -0.0484  0.0300  0.0019  ...   0.0955  0.0303 -0.0358
 -0.0617  0.0519  0.0188  ...   0.1229  0.0439 -0.0596
 -0.0571  0.0614  0.0280  ...   0.1361  0.0515 -0.0690
           ...             ⋱             ...          
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000

( 2 ,.,.) = 
 -0.0577  0.0260  0.0096  ...   0.0966  0.0200 -0.0346
 -0.0654  0.0336  0.0058  ...   0.1337  0.0353 -0.0632
 -0.0649  0.0410  0.0046  ...   0.1393  0.0307 

In [206]:
batch_size = 3
max_length = 3
hidden_size = 10
n_layers =1

# container
batch_in = torch.zeros((batch_size, 1, max_length))

#data
vec_1 = torch.FloatTensor([[1, 2, 3]])
vec_2 = torch.FloatTensor([[1, 2, 0]])
vec_3 = torch.FloatTensor([[1, 0, 0]])

batch_in[0] = vec_1
batch_in[1] = vec_2
batch_in[2] = vec_3

batch_in = Variable(batch_in)

seq_lengths = [3,2,1] # list of integers holding information about the batch size at each sequence step

# pack it
pack = torch.nn.utils.rnn.pack_padded_sequence(batch_in, seq_lengths, batch_first=True)
print(pack)

rnn = nn.RNN(max_length, hidden_size, n_layers, batch_first=True) 
h0 = Variable(torch.randn(n_layers, batch_size, hidden_size))

#forward 
out, _ = rnn(pack, h0)

# unpack
unpacked, unpacked_len = torch.nn.utils.rnn.pad_packed_sequence(out)

print("\n", unpacked)

PackedSequence(data=Variable containing:
 1  2  3
 1  2  0
 1  0  0
[torch.FloatTensor of size 3x3]
, batch_sizes=[3])

 Variable containing:
(0 ,.,.) = 

Columns 0 to 8 
   0.8215 -0.0527  0.7637  0.9655 -0.4354  0.5745  0.7437  0.8519  0.8935
  0.6907  0.5297  0.2629  0.0165 -0.3369 -0.4116  0.5638  0.7357  0.9065
  0.5517  0.4149  0.0342  0.0964  0.3720  0.7712  0.7045  0.1289 -0.7368

Columns 9 to 9 
  -0.9333
  0.3643
 -0.8083
[torch.FloatTensor of size 1x3x10]

